## 1. Aufbereitung der Urteilstexte und Vorbereitung des Analyse-Datensatzes

In diesem Abschnitt werden die eingelesenen OpenJur-Urteilstexte weiterverarbeitet und strukturiert. Ziel ist es, relevante Textbestandteile wie den Kopfbereich und den Tenor zu extrahieren, Landgerichtsurteile zu identifizieren und die Daten schließlich in ein geeignetes JSONL-Format für die nachgelagerte automatische Analyse zu überführen.


### 1.1 Import der benötigten Bibliotheken

Zu Beginn werden die für die weitere Verarbeitung erforderlichen Python-Bibliotheken importiert. Diese umfassen Funktionen für Dateizugriffe, reguläre Ausdrücke, Datenverarbeitung mit Pandas sowie den Export der Ergebnisse im JSON-Format.

In [1]:
#Import
import os
import re
import json
import pandas as pd

### 1.2 Einlesen der Urteilstexte und Aufbau des DataFrames

In diesem Schritt werden alle zuvor identifizierten Textdateien zeilenweise eingelesen. Für jede Datei wird eine eindeutige Fall-ID aus dem Dateinamen erzeugt und gemeinsam mit dem vollständigen Text in einem Pandas-DataFrame gespeichert. Der DataFrame bildet die zentrale Datenstruktur für die weitere Analyse.

In [2]:
# (.txt) Dateien einlesen
DATA_DIR = "../data/Gerichtsurteile_Openjur" 
files = [f for f in os.listdir(DATA_DIR) if f.lower().endswith(".txt")]

print("Pfad:", os.path.abspath(DATA_DIR))
print("Anzahl .txt:", len(files))
print("Erste 10 Dateien:", files[:10])


Pfad: c:\Users\humme\OneDrive\Dokumente\Uni Ulm\ds_law\backend\data\Gerichtsurteile_Openjur
Anzahl .txt: 2375
Erste 10 Dateien: ['2090187.txt', '2112111.txt', '2112115.txt', '2112117.txt', '2112118.txt', '2112119.txt', '2112121.txt', '2112123.txt', '2124977.txt', '2126821.txt']


---

## 2. Textvorverarbeitung und Extraktion zentraler Urteilsbestandteile

In diesem Abschnitt werden die eingelesenen Urteilstexte weiterverarbeitet, um für die nachfolgende Analyse relevante Textbestandteile gezielt zu extrahieren. Hierzu zählen insbesondere ein begrenzter Kopfbereich zur Voranalyse sowie der Tenor als Kern der gerichtlichen Entscheidung. Die strukturierte Aufbereitung dieser Textsegmente bildet die Grundlage für Filter-, Klassifikations- und Extraktionsschritte in den folgenden Abschnitten.

### 2.1 Erzeugung eines Kopfbereichs zur Voranalyse

Da relevante Metadaten wie Gericht, Entscheidungsart und Datum typischerweise am Anfang eines Urteilstextes stehen, wird ein begrenzter Kopfbereich (`head`) aus den ersten Zeichen des Dokuments extrahiert. Dieser verkürzte Textausschnitt dient als effizienter Suchraum für Filter- und Klassifikationsschritte.


In [3]:
rows = []
for fn in files:
    case_id = fn.replace(".txt", "")
    path = os.path.join(DATA_DIR, fn)
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        text = f.read()
    rows.append({"case_id": case_id, "text": text})

df = pd.DataFrame(rows)
print("Gesamt eingelesen:", len(df))


Gesamt eingelesen: 2375


In [6]:
HEAD_CHARS = 8000
df["head"] = df["text"].astype(str).str.slice(0, HEAD_CHARS)

print("Head-Länge (Beispiel):", len(df.loc[0, "head"]))


Head-Länge (Beispiel): 8000


### 2.2 Extraktion des Tenors

Der Tenor enthält die eigentliche gerichtliche Entscheidung und ist daher für die inhaltliche Bewertung besonders relevant. Mithilfe regulärer Ausdrücke wird der Textabschnitt zwischen der Überschrift „Tenor“ und den nachfolgenden Abschnitten (z. B. „Tatbestand“ oder „Gründe“) extrahiert und in einer separaten Spalte gespeichert.

In [7]:
def extract_tenor(text: str) -> str:
    if not isinstance(text, str):
        return ""
    m_start = re.search(r"\bTenor\b", text, flags=re.IGNORECASE)
    if not m_start:
        return ""
    start = m_start.end()
    m_end = re.search(
        r"\b(Tatbestand|Gründe|Gruende|Entscheidungsgründe|Entscheidungsgruende)\b",
        text[start:],
        flags=re.IGNORECASE
    )
    end = start + m_end.start() if m_end else min(len(text), start + 8000)
    return text[start:end].strip()

df["tenor"] = df["text"].apply(extract_tenor)

print("Tenor vorhanden:", (df["tenor"].str.len() > 0).sum(), "von", len(df))

Tenor vorhanden: 2362 von 2375


### 2.3 Identifikation von Landgerichtsurteilen (LG)

Im nächsten Schritt werden die Urteile anhand des Kopfbereichs danach gefiltert, ob es sich um Entscheidungen eines Landgerichts handelt. Dazu wird geprüft, ob charakteristische Begriffe wie „Landgericht“ oder die Abkürzung „LG“ im Kopfbereich vorkommen. Auf dieser Grundlage wird eine boolesche Variable erzeugt, die zur Selektion der relevanten Fälle dient.

In [10]:
# Wir suchen nach der Zeile, die mit "Einfach" beginnt, gefolgt von "LG"
# Der Regex r"Einfach\s*\n\s*LG" stellt sicher, dass LG direkt darunter steht
pattern_zitierung_lg = r"Einfach\s*\n\s*LG"

# Wir wenden das auf die Spalte an, die den Kopftext enthält
df["is_landgericht"] = df["head"].str.contains(pattern_zitierung_lg, regex=True, na=False)

# Jetzt erstellen wir den sauberen Dataframe
df_lg = df[df["is_landgericht"] == True].copy()

print("-" * 40)
print(f"✅ Echte LG-Urteile (über Zitierzeile): {len(df_lg)}")
print("-" * 40)

----------------------------------------
✅ Echte LG-Urteile (über Zitierzeile): 1189
----------------------------------------


### 2.4 Segmentierung der Urteile in juristische Abschnitte
Für die spätere Extraktion werden die Urteile in juristisch sinnvolle Teile zerlegt: Rubrum, Tenor, Tatbestand und Entscheidungsgründe. Dadurch kann das Modell gezielt relevante Passagen verarbeiten.


In [12]:
def split_judgment(text):
    """
    Teilt ein Urteil in Rubrum, Tenor, Tatbestand und Entscheidungsgründe auf.
    """
    segments = {
        "rubrum": "",
        "tenor": "",
        "tatbestand": "",
        "entscheidungsgruende": ""
    }
    
    # Muster für die Abschnittsüberschriften
    # Das Rubrum ist alles vor dem Tenor
    m_tenor = re.search(r"\bTenor\b", text, re.IGNORECASE)
    m_tatbestand = re.search(r"\bTatbestand\b", text, re.IGNORECASE)
    m_gruende = re.search(r"\b(Entscheidungsgründe|Gründe)\b", text, re.IGNORECASE)
    
    if m_tenor:
        segments["rubrum"] = text[:m_tenor.start()].strip()
        
        # Tenor bis Tatbestand
        if m_tatbestand:
            segments["tenor"] = text[m_tenor.end():m_tatbestand.start()].strip()
            
            # Tatbestand bis Gründe
            if m_gruende:
                segments["tatbestand"] = text[m_tatbestand.end():m_gruende.start()].strip()
                segments["entscheidungsgruende"] = text[m_gruende.end():].strip()
            else:
                segments["tatbestand"] = text[m_tatbestand.end():].strip()
        else:
            # Falls kein Tatbestand gefunden wird, Tenor bis zum Ende oder Gründen
            if m_gruende:
                segments["tenor"] = text[m_tenor.end():m_gruende.start()].strip()
                segments["entscheidungsgruende"] = text[m_gruende.end():].strip()
            else:
                segments["tenor"] = text[m_tenor.end():].strip()
                
    return segments

# Beispielanwendung auf den Dataframe
df_lg['segments'] = df_lg['text'].apply(split_judgment)

### 2.5 Aufbereitung der Textsegmente und Definition des Analyse-Prompts
Um API- und Token-Limits zu berücksichtigen, werden OpenJur-spezifische Navigationselemente aus dem Rubrum entfernt und alle Abschnitte in ihrer Länge begrenzt. Auf Basis dieser vorverarbeiteten Textsegmente wird ein standardisierter Prompt generiert, der die Extraktion der abgestimmten Variablen im JSON-Format steuert.


In [36]:
def clean_rubrum(rubrum: str) -> str:
    if not isinstance(rubrum, str):
        return ""

    blacklist = [
        "rechtsprechung", "aktuell", "trending", "filter",
        "über openjur", "spenden", "api", "hilfe",
        "startseite", "bundesland", "gerichtsbarkeit",
        "impressum", "datenschutz", "nutzungsbedingungen",
        "fachzeitschriften", "suchen", "changelog", "einfach",
        "json", "bibtex", "ris"
    ]

    lines = []
    for line in rubrum.splitlines():
        l = line.strip().lower()
        if not l:
            continue
        if any(b in l for b in blacklist):
            continue
        lines.append(line.strip())

    return "\n".join(lines[:3])   

def slim_segments(segments):
    return {
        "rubrum": clean_rubrum(segments.get("rubrum") or "")[:2500],
        "tenor": (segments.get("tenor") or "")[:4000],
        "tatbestand": (segments.get("tatbestand") or "")[:3500],
        "entscheidungsgruende": (segments.get("entscheidungsgruende") or "")[:7000],
    }

def get_gemini_prompt(segments):
    """
    Erstellt den finalen Prompt basierend auf den Urteilssegmenten.
    """
    s = slim_segments(segments)   # <--- DAS ist der entscheidende Schritt

    prompt = f"""
Analysiere die folgenden Abschnitte eines Gerichtsurteils zum Dieselskandal und extrahiere die Variablen präzise als JSON-Liste. 

### URTEILS-BESTANDTEILE:
RUBRUM (Kopfbereich mit Gericht & Datum): 
{s['rubrum']}

TENOR (Ergebnis): 
{s['tenor']}

TATBESTAND (Sachverhalt): 
{s['tatbestand']}

ENTSCHEIDUNGSGRÜNDE (Rechtliche Würdigung): 
{s['entscheidungsgruende']}

### EXTRAKTIONS-AUFGABE:
Extrahiere folgende Variablen (bei Nichtfinden 'null' angeben):

1. **Input-Variablen (Features):**
   - Dieselmotor_Typ
   - Art_Abschalteinrichtung
   - KBA_Rueckruf
   - Fahrzeugstatus
   - Fahrzeugmodell_Baureihe
   - Update_Status
   - Kilometerstand_Kauf
   - Kilometerstand_Klageerhebung
   - Erwartete_Gesamtlaufleistung
   - Kaufdatum
   - Uebergabedatum
   - Datum_Klageerhebung
   - Nachweis_Aufklaerung
   - Beklagten_Typ
   - Datum_Urteil
   - Kaufpreis
   - Nacherfuellungsverlangen_Fristsetzung
   - Klageziel
   - Rechtsgrundlage

2. **Zielvariablen (Labels):**
   - LABEL_Anspruch_Schadensersatz
   - LABEL_Schadensersatzhoehe_Betrag
   - LABEL_Schadensersatzhoehe_Range

### AUSGABEFORMAT:
Antworte NUR mit einem validen JSON-Objekt in einer Liste:
[{{
  "case_id": "...",
  "Dieselmotor_Typ": null,
  "Art_Abschalteinrichtung": null,
  "KBA_Rueckruf": null,
  "Fahrzeugstatus": null,
  "Fahrzeugmodell_Baureihe": null,
  "Update_Status": null,
  "Kilometerstand_Kauf": null,
  "Kilometerstand_Klageerhebung": null,
  "Erwartete_Gesamtlaufleistung": null,
  "Kaufdatum": null,
  "Uebergabedatum": null,
  "Datum_Klageerhebung": null,
  "Nachweis_Aufklaerung": null,
  "Beklagten_Typ": null,
  "Datum_Urteil": null,
  "Kaufpreis": null,
  "Nacherfuellungsverlangen_Fristsetzung": null,
  "Klageziel": null,
  "Rechtsgrundlage": null,
  "LABEL_Anspruch_Schadensersatz": null,
  "LABEL_Schadensersatzhoehe_Betrag": null,
  "LABEL_Schadensersatzhoehe_Range": null
}}]
""".strip()
    return prompt


### 2.7 Erstellung eines Pilot-Inputs im JSONL-Format (Batch-Input):
Zur technischen Validierung der Analysepipeline wird ein Pilotdatensatz erzeugt, der eine begrenzte Anzahl von Landgerichtsurteilen umfasst. Für jedes ausgewählte Urteil werden die zuvor definierten Textsegmente extrahiert, zu einem standardisierten Analyse-Prompt zusammengeführt und im JSONL-Format gespeichert. Diese Pilotdatei dient als Testeingabe für die nachgelagerte Verarbeitung über die Gemini-API, bevor eine Skalierung auf den vollständigen Datensatz erfolgt.

In [44]:
PILOT_N = 10
pilot_path = "gemini_batch_input_pilot_10.jsonl"

with open(pilot_path, "w", encoding="utf-8") as f:
    for _, row in df_lg.head(PILOT_N).iterrows():
        segments = split_judgment(row["text"])
        full_prompt = get_gemini_prompt(segments)

        payload = {
            "custom_id": f"case_{row['case_id']}",
            "contents": [{
                "role": "user",
                "parts": [{"text": full_prompt}]
            }]
        }
        f.write(json.dumps(payload, ensure_ascii=False) + "\n")

print("✅ Pilot erstellt:", pilot_path)


✅ Pilot erstellt: gemini_batch_input_pilot_10.jsonl


Der Code dient der inhaltlichen und technischen Validierung der erzeugten Pilotdatei. Hierzu wird der erste Eintrag der JSONL-Datei geladen und exemplarisch ausgegeben, um Struktur, Inhalt und Länge des generierten Analyse-Prompts zu überprüfen

In [43]:
with open("gemini_batch_input_pilot_10.jsonl", "r", encoding="utf-8") as f:
    first = json.loads(f.readline())

print(first["custom_id"])
print(first["contents"][0]["parts"][0]["text"][:800])
print("Prompt-Länge:", len(first["contents"][0]["parts"][0]["text"]))


case_2090187
Analysiere die folgenden Abschnitte eines Gerichtsurteils zum Dieselskandal und extrahiere die Variablen präzise als JSON-Liste. 

### URTEILS-BESTANDTEILE:
RUBRUM (Kopfbereich mit Gericht & Datum): 
Rechtsgebiet
Gericht
Informationen

TENOR (Ergebnis): 
I. Die Klage wird abgewiesen.II. Der Kläger hat die Kosten des Rechtsstreits zu tragen.III. Das Urteil ist gegen Sicherheitsleistung in Höhe des 1,1-fachen des zu vollstreckenden Betrags vorläufig vollstreckbar.IV. Der Streitwert wird auf 31.234,00 € festgesetzt.

TATBESTAND (Sachverhalt): 
Der Kläger begehrt Lieferung eines mangelfreien Pkw.Der Kläger erwarb von der Beklagten im Jahr 2014 einen Neuwagen VW Passat 2,0 l TDI für 31.234,00 €. Der Pkw ist von dem "VW-Abgasskandal" betroffen. Der Kläger hat die Beklagte im Jahr 2016 durch Anwa
Prompt-Länge: 8908


Client initialisiert


## 3.Übergabe an Gemini und Verarbeitung: 
In diesem Schritt wird die zuvor erzeugte Pilot-JSONL-Datei als Eingabe für die Gemini-API hochgeladen. Die Datei enthält strukturierte Analyseanfragen für mehrere Urteile und wird auf den Servern bereitgestellt, sodass sie anschließend im Rahmen einer Batch- oder sequenziellen Verarbeitung vom Sprachmodell verarbeitet werden kann. Der Upload stellt damit die technische Schnittstelle zwischen der lokalen Vorverarbeitung und der automatisierten Modellanalyse dar.

### 3.1 Initialisierung des API-Clients

In [45]:
from google import genai
import os

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
print("Client initialisiert")


Client initialisiert


### 3.2 Upload der JSONL-Datei

In [41]:
uploaded = client.files.upload(
    file="gemini_batch_input_pilot_10.jsonl",
    config={
        "display_name": "diesel-lg-pilot-10",
        "mime_type": "text/plain"
    }
)
print("Upload:", uploaded.name)


Upload: files/6v2xn4ygp6zq


In [ ]:
job = client.batches.create(
    model="models/gemini-2.5-flash",
    src=uploaded.name,
    config={"display_name": "diesel-lg-pilot-10"}
)

print("Batch gestartet:", job.name)


## 4. Aufbereitung der Modellantworten und Erstellung des Analyse-Datensatzes

Die in Abschnitt 2 aufbereiteten und gefilterten Urteilstexte liegen bereits in Form einer strukturierten JSONL-Datei vor.
Diese Datei dient als direkter Input für die nachgelagerte automatisierte Analyse und wird im Folgenden in das Batch-Verarbeitungssystem eingelesen.

(gemini_batch_input_NUR_LG.jsonl)

In [ ]:
# Hier Code einfügen

### 4.2 Übergabe des Analyse-Datensatzes an die Batch-API

Nach der in Abschnitt 2 beschriebenen Aufbereitung der Urteilstexte liegt der vollständige Analyse-Datensatz in Form einer strukturierten JSONL-Datei vor. Diese Datei dient in diesem Schritt als Eingabe für die automatisierte Verarbeitung durch ein großes Sprachmodell.

Die JSONL-Datei wird zunächst in das Batch-System hochgeladen. Anschließend wird ein Batch-Verarbeitungsjob gestartet, der die hochgeladene Datei als Eingabequelle verwendet. Für jedes enthaltene Dokument erzeugt das Modell eine strukturierte Antwort gemäß den im Prompt definierten Extraktionsvorgaben.

Als Ergebnis des Batch-Jobs stellt die API eine Ausgabedatei bereit, die die Modellantworten zu allen verarbeiteten Urteilen enthält. Diese Ausgabedatei liegt ebenfalls im JSONL-Format vor und bildet die Grundlage für die weitere Aufbereitung und Auswertung der Ergebnisse.


In [ ]:
# Hier Code einfügen

### 4.3 Aufbereitung der Batch-Ergebnisse und Erstellung des Analyse-Datensatzes

Die im vorherigen Schritt erzeugte Ausgabedatei des Batch-Jobs liegt zunächst als Rohdaten im JSONL-Format vor. Jede Zeile dieser Datei enthält die strukturierte Modellantwort zu einem einzelnen Landgerichtsurteil.

Diese Rohdaten werden lokal gespeichert und anschließend in ein tabellarisches Format überführt. Hierzu werden die relevanten Felder aus den JSON-Strukturen extrahiert und in einer einheitlichen Datenstruktur zusammengeführt, beispielsweise in Form einer CSV-Datei. 
Der so erzeugte Datensatz bildet die Grundlage für die weitere statistische Auswertung und Analyse in den folgenden Abschnitten.

In [ ]:
# Hier Code einfügen

---

## 5. Datenaufbereitung

In diesem Abschnitt werden die im vorherigen Schritt erzeugten Analyseergebnisse weiterverarbeitet und für die nachgelagerte statistische Auswertung vorbereitet. Grundlage hierfür ist der aus den Batch-Ergebnissen abgeleitete strukturierte Datensatz, der die vom Sprachmodell extrahierten Informationen zu den einzelnen Urteilen enthält.

Ziel der Datenaufbereitung ist es, die extrahierten Merkmale in eine konsistente, auswertbare Form zu überführen, fehlende oder uneinheitliche Angaben zu behandeln und die Zielvariablen für die spätere Analyse eindeutig zu definieren.

In [ ]:
import pandas as pd
import re
import spacy
import json
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Setup: Spezialisiertes deutsches Sprachmodell laden
try:
    nlp = spacy.load("de_core_news_lg")
except Exception:
    print("Bitte installiere das spacy Modell: python -m spacy download de_core_news_lg")

# --- 2. JURISTISCHE TEXTVORVERARBEITUNG ---
def legal_preprocess(text):
    """
    Bereitet juristische Texte auf, indem Rauschen entfernt wird, 
    während rechtlich relevante Zahlen und Kontexte geschützt werden.
    """
    if not isinstance(text, str) or not text:
        return ""

    # NEU: START DES URTEILS FINDEN (Rauschschnitt Anfang) ---
    # Wir schneiden Webseiten-Menüs ("trending", "suche" etc.) weg
    start_keywords = ["tenor", "entscheidungsgründe", "tatbestand", "urteil", "beschluss", "endurteil"]
    text_lower_start = text.lower()
    
    # Finde die früheste Position eines der Keywords
    found_positions = [text_lower_start.find(kw) for kw in start_keywords if text_lower_start.find(kw) != -1]
    if found_positions:
        text = text[min(found_positions):]

    # NEU: ENDE DES URTEILS FINDEN (Rauschschnitt Ende) ---
    # Wir schneiden Impressum und Footer weg
    end_keywords = ["impressum", "nutzungsbedingungen", "nach oben", "datenschutz"]
    text_lower_end = text.lower()
    for ekw in end_keywords:
        e_pos = text_lower_end.find(ekw)
        if e_pos != -1:
            text = text[:e_pos]
            break

    # 1. Bereinigung von Rauschen (HTML-Tags, Sonderzeichen)
    text = re.sub(r'<.*?>', ' ', text)

    # 2. Schutz von Zahlen & Paragraphen (Platzhalter statt Löschen)
    # Euro-Beträge schützen
    text = re.sub(r'\d{1,3}(?:\.\d{3})*(?:,\d+)?\s*(?:EUR|€|Euro)', ' PLATZHALTER_BETRAG ', text)
    # Paragraphen schützen
    text = re.sub(r'§+\s*\d+[a-z]?\s*(?:\w+)?', ' PLATZHALTER_PARAGRAPH ', text)
    # Jahreszahlen schützen
    text = re.sub(r'\b(19|20)\d{2}\b', ' PLATZHALTER_JAHR ', text)

    # 3. Kleinschreibung zur Reduktion der Varianz
    text = text.lower()

    # 4. Tokenisierung und Lemmatisierung mit SpaCy
    doc = nlp(text)
    
    # 5. Kontextsensitive Stoppwort-Entfernung
    # Wichtige juristische Negationen schützen
    protected_negations = {"nicht", "kein", "ohne", "gegen", "trotz"}
    custom_stop_words = nlp.Defaults.stop_words - protected_negations
    
    # Extraktion der Lemmata (Grundformen)
    tokens = [
        token.lemma_ for token in doc 
        if token.text not in custom_stop_words 
        and not token.is_punct 
        and not token.is_space
        and len(token.text) > 1 # Token mit Länge 1 entfernen
    ]
    
    return " ".join(tokens)

# --- 2.5 HILFSFUNKTIONEN: Simulation + echtes Batch lesen ---
def get_llm_text(r: dict) -> str:
    # Simulation (simulated_batch_output.jsonl)
    if "text" in r:
        return r["text"]
    # Echtes Batch (später)
    if "response" in r:
        return r["response"]["body"]["choices"][0]["message"]["content"]
    raise KeyError("Unbekanntes Ergebnisformat (kein 'text' und kein 'response').")

def parse_llm_json(text: str) -> dict:
    # Entfernt ```json ... ``` falls vorhanden
    text = re.sub(r"^```json\s*|\s*```$", "", text.strip(), flags=re.MULTILINE)
    # Falls außenrum Text steht: ersten JSON-Block extrahieren
    m = re.search(r"(\{.*\})", text, flags=re.DOTALL)
    if m:
        text = m.group(1)
    return json.loads(text)

# --- 3. MERGING DER DATEN (URTEILE + EXTRAKTIONEN) ---
def merge_and_finalize(judgment_file, batch_results_file):
    """
    Führt die ursprünglichen Urteilstexte mit den Gemini-Extraktionen zusammen.
    """
    # 1. Laden der aufbereiteten LG-Urteile
    df_judgments = pd.read_json(judgment_file, lines=True)
    df_judgments['case_id'] = df_judgments['custom_id'].str.replace('case_', '')

    # 2. Laden der Gemini-Batch-Ergebnisse
    with open(batch_results_file, 'r', encoding='utf-8') as f:
        results = [json.loads(line) for line in f]
    
    extracted_rows = []
    for r in results:
        try:
            case_id = r['custom_id'].replace('case_', '')
            llm_text = get_llm_text(r)
            content = parse_llm_json(llm_text)
            content['case_id'] = case_id
            extracted_rows.append(content)
        except Exception:
            continue
            
    df_extracted = pd.DataFrame(extracted_rows)

    # 3. Zusammenführung über case_id 
    df_final = pd.merge(df_judgments, df_extracted, on='case_id', how='inner')

    # 4. Textverarbeitung anwenden
    print("Starte Textvorverarbeitung...")
    df_final['cleaned_text'] = df_final['text'].apply(legal_preprocess)

    return df_final

# --- 4. MODELL-VORBEREITUNG (TF-IDF) ---
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),   # Bigramme erhalten Wortzusammenhänge
    max_features=1000,    # Reduktion der Komplexität
    min_df=5              # Seltene Begriffe ignorieren
)

In [ ]:
RESULTS_FILE = "simulated_batch_output.jsonl"

In [ ]:
df_final = merge_and_finalize(
    judgment_file="lg_judgments.jsonl",
    batch_results_file=RESULTS_FILE
)

print(df_final.shape)
df_final.head()


In [19]:
##Test der Datenvorverarbeitung auf einzelne Datei
import os
from pathlib import Path

# Pfad zu den Gerichtsurteilen
base_path = Path.cwd().parent 
pfad_zu_urteilen = base_path / "data" / "Gerichtsurteile_Openjur"

# Test mit einer Datei aus deiner Liste
test_datei_name = "2090187.txt" 
voller_pfad = pfad_zu_urteilen / test_datei_name

try:
    with open(voller_pfad, "r", encoding="utf-8") as f:
        original_text = f.read()
    
    # Schneller Test mit 2.000 Zeichen
    test_schnipsel = original_text[:2000]
    gereinigter_text = legal_preprocess(test_schnipsel)
    
    print(f"✅ Datei gefunden: {test_datei_name}")
    print("-" * 40)
    print(gereinigter_text[:500] + "...")

except FileNotFoundError:
    print(f"❌ Pfad falsch. Er sucht hier: {voller_pfad}")

✅ Datei gefunden: 2090187.txt
----------------------------------------
Endurteil 23.05. Platzhalter_jahr 21 658 16 Zitierung einfach lg Bayreuth Endurteil 23.05. Platzhalter_jahr 21 658 16 M. Fundstelle lg Bayreuth Endurteil 23.05. Platzhalter_jahr 21 658 16 Openjur Platzhalter_jahr 9516 Bibtex ris json lg Bayreuth Endurteil 23.05. Platzhalter_jahr 21 658 16 fundstell Openjur Platzhalter_jahr 9516 Rechtskraft Option Zitierung Version Tenor i. Klage abgewiesen.ii Kläger Kosten rechtsstreit tragen.iii Urteil gegen Sicherheitsleistung Höhe 1,1-fache vollstreckend betr...


Nachweis der Daten-Pipeline (Schritt 1 & 2)
Pfad-Validierung: Python-Skript versteht die Ordnerstruktur und kann die über 2.300 .txt-Dateien einlesen. 
Filter-Logik: Die Statistik zeigt, dass dein Code erfolgreich zwischen Landgerichten (LG) und Oberlandesgerichten (OLG) unterscheidet. 
Rauschschnitt (Slicing): Die Menüführung der Webseite (Trending, Spenden, Suche) ist abgeschnitten
Abstraktion durch Platzhalter: Dass PLATZHALTER_JAHR auftaucht, bestätigt, dass deine Regex-Logik funktioniert. Das Modell wird dadurch gezwungen, juristische Muster (wie „Klage abgewiesen“) zu lernen, statt sich an unwichtigen spezifischen Zahlen festzubeißen.

## 6. Analyse und Auswertung